In [20]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import os

# CSV file path
csv_file = "data.csv"

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Real-Time Sentiment Dashboard"

# Layout for the dashboard
app.layout = dbc.Container([
    html.H1("Real-Time Sentiment Analysis Dashboard", className="text-center my-4"),

    dcc.Interval(id="data-update-interval", interval=5000, n_intervals=0),  # Refresh every 5 seconds

    dbc.Row([
        dbc.Col([
            html.H3("Histogram: Sentiment Counts", className="text-center"),
            dcc.Graph(id="sentiment-histogram")
        ], width=6),

        dbc.Col([
            html.H3("Pie Chart: Sentiment Distribution", className="text-center"),
            dcc.Graph(id="sentiment-piechart")
        ], width=6)
    ]),

    html.H3("Sentiment Counts Table", className="text-center mt-4"),
    html.Div(id="sentiment-data-table"),

    html.H3("Post Titles and Their Sentiments", className="text-center mt-4"),
    html.Div(id="titles-sentiment-table"),

    html.H3("Topics Generating the Most Emotional Responses", className="text-center mt-4"),
    html.Div(id="topics-sentiment-table")
], fluid=True)

# Callback to update dashboard elements
@app.callback(
    [Output("sentiment-histogram", "figure"),
     Output("sentiment-piechart", "figure"),
     Output("sentiment-data-table", "children"),
     Output("titles-sentiment-table", "children"),
     Output("topics-sentiment-table", "children")],
    [Input("data-update-interval", "n_intervals")]
)
def update_dashboard(n_intervals):
    if not os.path.exists(csv_file):
        return {}, {}, "No data available. Waiting for updates...", "No data available. Waiting for updates...", "No data available. Waiting for updates..."

    # Load data from CSV file
    data = pd.read_csv(csv_file)

    # Sentiment counts
    sentiment_counts = data['selftext_sentiment'].value_counts()

    # Define consistent colors
    sentiment_colors = {"Positive": "green", "Neutral": "blue", "Negative": "red"}
    colors = [sentiment_colors.get(sentiment, "gray") for sentiment in sentiment_counts.index]

    # Histogram
    histogram = {
        "data": [
            {
                "x": sentiment_counts.index,
                "y": sentiment_counts.values,
                "type": "bar",
                "marker": {"color": colors},
            }
        ],
        "layout": {
            "title": "Sentiment Counts",
            "xaxis": {"title": "Sentiment"},
            "yaxis": {"title": "Count"}
        }
    }

    # Pie Chart
    piechart = {
        "data": [
            {
                "values": sentiment_counts.values,
                "labels": sentiment_counts.index,
                "type": "pie",
                "hole": 0.4,
                "marker": {"colors": colors}
            }
        ],
        "layout": {
            "title": "Sentiment Distribution"
        }
    }

    # Data Table with sentiment counts
    sentiment_data = pd.DataFrame({
        "Sentiment": sentiment_counts.index,
        "Count": sentiment_counts.values
    })
    sentiment_data.loc["Total"] = ["Total", sentiment_counts.sum()]  # Add total row
    sentiment_table = dbc.Table.from_dataframe(
        sentiment_data,
        striped=True, bordered=True, hover=True
    )

    # Titles and Sentiments Table
    titles_sentiment = data[['title', 'selftext_sentiment']].drop_duplicates().head(10)  # Show 10 different titles
    titles_sentiment_table = dbc.Table.from_dataframe(
        titles_sentiment,
        striped=True, bordered=True, hover=True
    )

    # Topics and Sentiments Table
    if 'query' in data.columns:
        topics_sentiment = data.groupby(['query', 'selftext_sentiment']).size().unstack(fill_value=0)
        topics_sentiment['Total'] = topics_sentiment.sum(axis=1)  # Add total column
        topics_sentiment = topics_sentiment.sort_values(by='Total', ascending=False).head(10)  # Show top 10 topics
        topics_sentiment_table = dbc.Table.from_dataframe(
            topics_sentiment.reset_index(),
            striped=True, bordered=True, hover=True
        )
    else:
        topics_sentiment_table = "No query data available."

    return histogram, piechart, sentiment_table, titles_sentiment_table, topics_sentiment_table

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=3077)
